In [1]:
import math
import pandas as pd

In [2]:
#initial conditions
c = math.pi/180 #conversion factor
g = 9.81 #gravity

#acceleration
x_1 = 0
y_1 = 0

#dimension
L_1 = 0.48
L_2 = 0.44
L_3 = 0.55
L_lb1 = 0.12
L_a = 0.69

#mass
M = 180 #total
M_b = 300 #bar

M_1 = M*0.043
M_2 = M*0.11
M_3 = M*0.53
M_b1 = 0
M_a = M*0.049
M_a += M*0.07 #adding head

#proximal distance
P_1 = 0.5
P_2 = 0.5
P_3 = 0.5
P_lb1 = 0.5
P_a = 0.5

#inertia
I_1 = 0.5*M_1*L_1**2
I_2 = 0.5*M_2*L_2**2
I_3 = 0.5*M_3*L_3**2
I_lb1 = 0.5*M_b1*L_lb1**2
I_a = 0.5*M_a*L_a**2

In [3]:
class angular_acceleration_velocity:

    def a_i(O__i, O_i, t_i):
        return(2*(O__i-O_i)/(t_i**2))

    def w_i(O__i, O_i, a_i):
        return(2*a_i*(O__i-O_i))

In [4]:
class accelerations:
    
    #general (joint)
    def x__i(x_i, L_i, a_i, w_i, theta): #x__i = x_(i+1)
        return(x_i-L_i*((a_i)*math.sin(theta)+w_i**2*math.cos(theta)))

    def y__i(y_i, L_i, a_i, w_i, theta):
        return(y_i+L_i*((a_i)*math.cos(theta)-w_i**2*math.sin(theta)))

    #general (limb - distal)
    def x_c(x_i, P_i, L_i, a_i, w_i, theta):
        return(x_i-(1-P_i)*L_i*((a_i)*math.sin(theta)+w_i**2*math.cos(theta)))

    def y_c(y_i, P_i, L_i, a_i, w_i, theta):
        return(y_i+(1-P_i)*L_i*((a_i)*math.cos(theta)-w_i**2*math.sin(theta)))

    #general (limb - proximal)
    def x__c(x_i, P_i, L_i, a_i, w_i, theta):
        return(x_i-P_i*L_i*((a_i)*math.sin(theta)+w_i**2*math.cos(theta)))

    def y__c(y_i, P_i, L_i, a_i, w_i, theta):
        return(y_i+P_i*L_i*((a_i)*math.cos(theta)-w_i**2*math.sin(theta)))

In [5]:
class forces:

    #segment 1,2
    def Fx_12(Fx__i,M_i,x_c):
        return(Fx__i - M_i*x_c)

    def Fy_12(Fy__i,m_i,y_c):
        return(Fy__i + m_i*(y_c+g))

    #torso+head - "simplification"
    def Fx_3(Fx_lb1,m_3,x_c):
        return(Fx_lb1 - m_3*x_c)

    def Fy_3(Fy_lb1,m_3,y_c):
        return(Fy_lb1 + m_3*(y_c+g))

    #lb1
    def Fx_lb1(Fx_a):
        return(Fx_a)

    def Fy_lb1(Fy_a):
        return(Fy_a)

    #arm - "simplified"
    def Fx_a(Fx_b, m_5, x_c):
        return(Fx_b - m_5*x_c)

    def Fy_a(Fy_b, m_5, y_c):
        return(Fy_b + m_5*(y_c+g))

    #bar
    def Fx_b(m_b, x_b):
        return(-m_b*x_b)

    def Fy_b(m_b, y_b):
        return(m_b*(y_b+g))


In [6]:
class moments:

    def m_a(I_a, P_a, a_4, Fx_4, Fy_4, Fx_b, Fy_b, theta):
        return(I_a*a_4-Fx_4*P_a*L_a*math.sin(theta)-Fy_4*P_a*L_a*math.cos(theta)-Fx_b*(1-P_a)*L_a*math.sin(theta)-Fy_b*(1-P_a)*L_a*math.cos(theta))

    def m_lb1(I_lb1, P_lb1, L_lb1, a_lb1, Fx_lb1, Fy_lb1, Fx_4, Fy_4, theta):
        return(I_lb1*a_lb1-Fx_lb1*P_lb1*L_lb1*math.sin(theta)-Fy_lb1*P_lb1*L_lb1*math.cos(theta)-Fx_4*(1-P_lb1)*L_lb1*math.sin(theta)-Fy_4*(1-P_lb1)*L_lb1*math.cos(theta))

    def m_3(I_3, P_3, L_3, a_3, Fx_3, Fy_3, Fx_lb1, Fy_lb1, theta,m_lb1):
        return(I_3*a_3+Fx_3*P_3*L_3*math.sin(theta)+Fy_3*P_3*L_3*math.cos(theta)+Fx_lb1*(1+P_3)*L_3*math.sin(theta)+Fy_lb1*(1+P_3)*L_3*math.cos(theta)+m_lb1)

    def m_2(I_2, P_2, L_2, a_2, Fx_2, Fy_2, Fx_3, Fy_3, theta, m_3):
        return(I_2*a_2+Fx_2*(1-P_2)*L_2*math.sin(theta)+Fy_2*(1-P_2)*L_2*math.cos(theta)+Fx_3*P_2*L_2*math.sin(theta)+Fy_3*P_2*L_2*math.cos(theta)+m_3)
    
    def m_1(I_1, P_1, L_1, a_1, Fx_1, Fy_1, Fx_2, Fy_2, theta, m_2):
        return(I_1*a_1+Fx_1*(1-P_1)*L_1*math.sin(theta)+Fy_1*(1-P_1)*L_1*math.cos(theta)+Fx_2*P_1*L_1*math.sin(theta)+Fy_2*P_1*L_1*math.cos(theta)+m_2)

In [7]:
#interval 1

#angle
O_1 = 80*c
O_2 = 170*c
O_3 = 37*c
O_4 = 180*c

#angle'
O__1=88*c
O__2=141*c
O__3=40*c
O__4=180*c

#time
t_i = 0.5 

#accelerations (what angle for acceleration eqns...)
a_1 = angular_acceleration_velocity.a_i(O__1, O_1, t_i)
w_1 = angular_acceleration_velocity.w_i(O__1, O_1, a_1)
x_1c = accelerations.x_c(x_1, P_1, L_1, a_1, w_1, O__1)
y_1c = accelerations.y_c(y_1, P_1, L_1, a_1, w_1, O__1)
x_2 = accelerations.x__i(x_1, L_1, a_1, w_1, O__1)
y_2 = accelerations.y__i(y_1, L_1, a_1, w_1, O__1)

a_2 = angular_acceleration_velocity.a_i(O__2, O_2, t_i)
w_2 = angular_acceleration_velocity.w_i(O__2, O_2, a_2)
x_2c = accelerations.x_c(x_2, P_2, L_2, a_2, w_2, O__2)
y_2c = accelerations.y_c(y_2, P_2, L_2, a_2, w_2, O__2)
x_3 = accelerations.x__i(x_2, L_2, a_2, w_2, O__2)
y_3 = accelerations.y__i(y_2, L_2, a_2, w_2, O__2)

a_3 = angular_acceleration_velocity.a_i(O__3, O_3, t_i)
w_3 = angular_acceleration_velocity.w_i(O__3, O_3, a_3)
x_3c = accelerations.x_c(x_3, P_3, L_3, a_3, w_3, O__3)
y_3c = accelerations.y_c(y_3, P_3, L_3, a_3, w_3, O__3)
x_lb1 = accelerations.x__i(x_3, L_3, a_3, w_3, O__3)
y_lb1 = accelerations.y__i(y_3, L_3, a_3, w_3, O__3)

a_lb1 = 0
w_lb1 = 0

a_4 = angular_acceleration_velocity.a_i(O__4, O_4, t_i)
w_4 = angular_acceleration_velocity.w_i(O__4, O_4, a_4)
x_lb1c = accelerations.x_c(x_lb1, P_lb1, L_lb1, a_4, w_4, O__4)
y_lb1c = accelerations.y_c(y_lb1, P_lb1, L_lb1, a_4, w_4, O__4)
x_a = accelerations.x__i(x_lb1, L_lb1, a_4, w_4, O__4)
y_a = accelerations.y__i(y_lb1, L_lb1, a_4, w_4, O__4)

x_b = x_a
y_b = y_a

#forces
Fx_b = forces.Fx_b(M_b, x_b)
Fy_b = forces.Fy_b(M_b, y_b)

Fy_a = forces.Fy_a(Fy_b,M_a,y_a)
Fx_a = forces.Fx_a(Fx_b,M_a,x_a)

Fx_lb1 = Fx_a #sus
Fy_lb1 = Fy_a

Fy_3 = forces.Fy_3(Fy_lb1,M_3,y_3c)
Fx_3 = forces.Fx_3(Fx_lb1,M_3,x_3c)

Fy_2 = forces.Fy_12(Fy_3,M_2,y_2c)
Fx_2 = forces.Fx_12(Fx_3,M_2,x_2c)

Fy_1 = forces.Fy_12(Fy_2,M_1,y_1c)
Fx_1 = forces.Fx_12(Fx_2,M_1,x_1c)

#moments
M_a = moments.m_a(I_a, P_a, a_4, Fx_a, Fy_a, Fx_b, Fy_b, O__4)
M_lb1 = moments.m_lb1(I_lb1, P_lb1, L_lb1, 0, Fx_lb1, Fy_lb1, Fx_a, Fy_a, 180) #sus?
M_3 = moments.m_3(I_3, P_3, L_3, a_3, Fx_3, Fy_3, Fx_lb1, Fy_lb1, O__3, M_lb1)
M_2 = moments.m_2(I_2, P_2, L_2, a_2, Fx_2, Fy_2, Fx_3, Fy_3, O__2, M_3)
M_1 = moments.m_1(I_1, P_1, L_1, a_1, Fx_1, Fy_1, Fx_2, Fy_2, O__1, M_2)

#output
d = {'x_acc':[x_1, x_2, x_3, x_lb1, x_a, x_b], 'y_acc':[y_1, y_2, y_3, y_lb1, y_a, y_b], 'x_force':[Fx_1, Fx_2, Fx_3, Fx_lb1, Fx_a, Fx_b], 'y_force':[Fy_1, Fy_2, Fy_3, Fy_lb1, Fy_a, Fy_b], 'moment':[M_1, M_2, M_3, M_lb1, M_a, 0]}

i = ['shank', 'thigh', 'torso', 'shoulder', 'arm', 'bar']

df = pd.DataFrame(d, i)

In [8]:
df

,x_acc,y_acc,x_force,y_force,moment
shank,0.000000,0.000000,-2638.279088,3017.356990,-2520.168201
thigh,-0.537468,-0.027963,-2640.359091,2941.535808,-1304.973730
torso,6.328837,-3.295656,-2583.024541,2780.201637,404.224595
shoulder,6.179939,-3.119852,-1986.355929,2150.347359,-36.537271
arm,6.179939,-3.119852,-1986.355929,2150.347359,1434.300153
bar,6.179939,-3.119852,-1853.981640,2007.044389,0.000000
